In [ ]:
!pip install pypdf sentence-transformers faiss-cpu langchain langchain-community
!apt-get install -y tesseract-ocr tesseract-ocr-ind
!pip install -q pytesseract pillow
!apt-get update -y
!apt-get install -y poppler-utils

<h1 style="color:'blue'">COnvert to image so we can do ocr</h1>

In [3]:
from pdf2image import convert_from_path

pdf_path = "/kaggle/input/datasetkesehatan/handbook-of-pharmaceutical-excipients-6th-ed-A.pdf"

images = convert_from_path(pdf_path, dpi=300)
print(f"Total pages: {len(images)}")


Total pages: 52


In [5]:
import pytesseract

text = ""

for i, img in enumerate(images):
    page_text = pytesseract.image_to_string(
        img,
        lang="eng",  
        config="--psm 6"
    )
    text += f"\n--- Page {i+1} ---\n"
    text += page_text

print(text[:1000])



--- Page 1 ---
SI Acacia
a i ry
1 Nonproprietary Names 9 Pharmacopeial Specifications
BP: Acacia The PhEur 6.3 provides monographs on acacia and spray-dried
JP: Acacia acacia, while the USP32—-NF27 describes acacia in a single
PhEur: Acacia monograph that encompasses tears, flakes, granules, powder, and
USP-NF: Acacia spray-dried powder. The USP32—NF27 also has a monograph on
: acacia syrup. The JP XV has monographs on acacia and powdered
acacia. See Table II.
2 Synonyms
Acaciae gummi; acacia gum; arabic gum; E414; gum acacia; gummi Table Il: Pharmacopeial specifications for acacia.
africanum; gum arabic; gummi arabicum; gummi mimosae; talha Jog JP XV PhEur 6.3 USP32-NF27
gum. Wentfientan
Identification + + +
Characters + + +
3 Chemical Name and CAS Registry Number Microbial limit - <10*%chu/g +
Acacia [9000-01-5] Water <] 7.0% <15.0% 5 <15.0%
<15.0%" <10.0%")  —
Total ash <4.0% <4.0% <4.0%
4 Empirical Formula and Molecular Weight Acie insoluble ash SOs <0.5% SSO mg /5g
Acacia is a co

In [6]:
import re

def clean_text(t):
    t = re.sub(r'\n{2,}', '\n', t) #coba remove space
    t = re.sub(r'[^\x00-\x7F]+', ' ', t) #remove x7 atau yang bukan ascii character kebetulan datasetnya cukup relevan
    return t.strip()

cleaned_text = clean_text(text)
print(cleaned_text[:1000])


--- Page 1 ---
SI Acacia
a i ry
1 Nonproprietary Names 9 Pharmacopeial Specifications
BP: Acacia The PhEur 6.3 provides monographs on acacia and spray-dried
JP: Acacia acacia, while the USP32 -NF27 describes acacia in a single
PhEur: Acacia monograph that encompasses tears, flakes, granules, powder, and
USP-NF: Acacia spray-dried powder. The USP32 NF27 also has a monograph on
: acacia syrup. The JP XV has monographs on acacia and powdered
acacia. See Table II.
2 Synonyms
Acaciae gummi; acacia gum; arabic gum; E414; gum acacia; gummi Table Il: Pharmacopeial specifications for acacia.
africanum; gum arabic; gummi arabicum; gummi mimosae; talha Jog JP XV PhEur 6.3 USP32-NF27
gum. Wentfientan
Identification + + +
Characters + + +
3 Chemical Name and CAS Registry Number Microbial limit - <10*%chu/g +
Acacia [9000-01-5] Water <] 7.0% <15.0% 5 <15.0%
<15.0%" <10.0%")   
Total ash <4.0% <4.0% <4.0%
4 Empirical Formula and Molecular Weight Acie insoluble ash SOs <0.5% SSO mg /5g
Acacia is a com

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=100
)

chunks = splitter.split_text(cleaned_text)
len(chunks)


712

In [10]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer(
    "BAAI/bge-m3",
    device="cuda"  
)
embeddings = embedder.encode(
    chunks,
    batch_size=16,                
    show_progress_bar=True,
    normalize_embeddings=True     
)


2025-12-18 02:43:34.225304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766025814.394377     188 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766025814.441423     188 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766025814.843276     188 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766025814.843310     188 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766025814.843313     188 computation_placer.cc:177] computation placer alr

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/45 [00:00<?, ?it/s]

In [11]:
import faiss
import numpy as np

dimension = embeddings.shape[1]

index = faiss.IndexFlatIP(dimension)
index.add(np.array(embeddings))

print("Total vectors:", index.ntotal)

Total vectors: 712


In [13]:
def retrieve(query, k=5):
    query = "Represent this sentence for searching relevant passages: " + query
    q_emb = embedder.encode(
        [query],
        normalize_embeddings=True
    )
    scores, indices = index.search(np.array(q_emb), k)
    
    return [(chunks[i], scores[0][idx]) for idx, i in enumerate(indices[0])]


In [14]:
results = retrieve("What is ACACIA?")
for r in results:
    print(r[1])
    print(r[0][:200])
    print("----")
#oke sih soalnya relevant dan similarity score nanti coiba kita gabungkan pakai jaccard similarity

0.51056814
Acacia is a complex, loose aggregate of sugars and hemicelluloses Arsenic     <3 ppm
with a molecular weight of approximately 240 000-580000. The Lead   - <0.001 %
aggregate consists essentially of an
----
0.46822992
the bark. The dried exudate is then collected, processed to remove   radiation on Reacia powder J Pharm P. Para ot os ; rn 45 93:
bark, sand, and other particulate matter, and graded. Various acacia 9
----
0.44865412
thus obtained. A spray-dried powder is also commercially available. Boca Raton, FL: CRC Press, 1992: 7-11. .
8 FAO/WHO. Evaluation of certain food additives and contaminants.
14 Safety Thirty-fifth re
----
0.4485557
--- Page 1 ---
SI Acacia
a i ry
1 Nonproprietary Names 9 Pharmacopeial Specifications
BP: Acacia The PhEur 6.3 provides monographs on acacia and spray-dried
JP: Acacia acacia, while the USP32 -NF27 de
----
0.4464891
8 Description ity increases rapidly (non-Newtonian rheology). Increasing
Acacia is available as white or yellowish-white th

In [17]:
results[0]

("Acacia is a complex, loose aggregate of sugars and hemicelluloses Arsenic     <3 ppm\nwith a molecular weight of approximately 240 000-580000. The Lead   - <0.001 %\naggregate consists essentially of an arabic acid nucleus to which are soy metals d    ~ <0.004%\nconnected calcium, magnesium, and potassium along with the vlore 'b extrin, anc agar   + +\n: annin-bearing gums + + +",
 np.float32(0.51056814))

In [24]:
#it seems like not clean on our text but it just an experiment
def build_prompt(contexts, question):
    context_text = "\n\n".join(
        [f"{c}" for c in contexts]
    )
    
    return f"""###Jawab pertanyaan berdasarkan konteks berikut.
    Jika jawabannya tidak ada di konteks, jawab: "Tidak ditemukan di dokumen".

    ###KONTEKS:
    {context_text}

    ###PERTANYAAN:
    {question}
    """


In [19]:
import faiss
import os

SAVE_DIR = "/kaggle/working/faiss_store"
os.makedirs(SAVE_DIR, exist_ok=True)

faiss_index_path = f"{SAVE_DIR}/index.faiss"

faiss.write_index(index, faiss_index_path)

print("FAISS index saved to:", faiss_index_path)


FAISS index saved to: /kaggle/working/faiss_store/index.faiss


In [20]:
import pickle

chunks_path = f"{SAVE_DIR}/chunks.pkl"

with open(chunks_path, "wb") as f:
    pickle.dump(chunks, f)

print("Chunks saved to:", chunks_path)

Chunks saved to: /kaggle/working/faiss_store/chunks.pkl


In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"   # otomatis ke GPU
)

llm.eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
  

In [25]:
def generate_answer(prompt, max_new_tokens=256):
    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to(llm.device)

    with torch.no_grad():
        output = llm.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.3,
            top_p=0.9,
            repetition_penalty=1.1
        )

    return tokenizer.decode(
        output[0],
        skip_special_tokens=True
    )

In [26]:
question = "jelaskan apa itu Acesulfame Potassium?"

contexts = retrieve(question, k=5)   

prompt = build_prompt(contexts, question)

answer = generate_answer(prompt)
print(answer)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


###Jawab pertanyaan berdasarkan konteks berikut.
    Jika jawabannya tidak ada di konteks, jawab: "Tidak ditemukan di dokumen".

    ###KONTEKS:
    ('Whistler RL. Industrial Gums. New York: Academic Press, 1959. 10 February 2009.\nAcesulfame Potassi\n1 Nonproprietary Names than sodium cyclamate.) It enhances flavor systems and can be\nBP: Acesulfame Potassium used to mask some unpleasant taste characteristics.\nPhEur: Acesulfame Potassium\nUSP-NF: Acesulfame Potassium 8 Description\nAcesulfame potassium occurs as a colorless to white-colored,', np.float32(0.6346382))

('Acesulfame potassium occurs as a colorless to white-colored,\n2 Synonyms odorless, crystalline powder with an intensely sweet taste.\nAcesulfame K; ace K; acesulfamum kalicum; E950; 6-methyl-3,4-\ndihydro-1,2,3-oxathiazin-4(3 H)-one-2,2-dioxide potassium _ salt; 9 Ph ial Specificati\npotassium 6-methyl-2,2-dioxo-oxathiazin-4-olate; Sunett; Sweet armacopelal specitications\nOne. See Table I.', np.float32(0.6144395))

("